
# Análise de Fatores de Risco para a Covid-19 

Neste projeto, serão analisados os fatores de risco para a Covid-19, levando em consideração doenças que são fatores de risco, idade, gênero e outras características.  

**Alunas: Eliane Dantas de Jesus e Natalia da Silva Costa**

### Imports necessários para a execução do projeto

In [1]:
import pandas as pd
import re
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

### Leitura do arquivo csv

In [2]:
df = pd.read_csv('Casos_e_obitos_ESP.csv', sep=';')

### Verificação da quantidade de registros

In [ ]:
num_registros = len(df)
print("Número total de registros:", num_registros)

### Verificação dos valores faltando por coluna

In [ ]:
valores_faltando = df.isnull().sum()
print("\nValores faltando por coluna:")
print(valores_faltando)

50.347302

### Verificação do número de linhas duplicadas

In [ ]:
linhas_duplicadas = df.duplicated().sum()
print("\nNúmero de linhas duplicadas:", linhas_duplicadas)

99.99

### Verificação de dados sujos

In [3]:
# Verificação do padrão da data
def verificar_padrao_data(data):
    if pd.isnull(data):  # Verificar se o valor é nulo
        return False  # Se for nulo, não é uma data suja
    # Verificar se a data possui 8 dígitos e o formato xx/xx/xxxx
    padrao = re.compile(r'^\d{2}/\d{2}/\d{4}$')
    if re.match(padrao, str(data)):  # Convertendo para string antes da verificação
        return True
    else:
        return False

Language    1098
dtype: int64

In [ ]:
# Verificação se é uma data suja
datas_sujas = []
for index, row in df.iterrows():
    if not verificar_padrao_data(row['Data_Inicio_Sintomas']):
        datas_sujas.append((index, row['Data_Inicio_Sintomas']))

# Imprimir as datas sujas
print("Datas sujas:")
for index, data in datas_sujas:
    print("Linha:", index, "- Data:", data)

### Estatísticas descritivas

In [6]:
print("\nEstatísticas descritivas:")
estatisticas_descritivas = df.describe()
print(estatisticas_descritivas)

Job    30
dtype: int64

In [8]:
# Calcular o IQR
Q1 = df['Idade'].quantile(0.25)
Q3 = df['Idade'].quantile(0.75)
IQR = Q3 - Q1

Job    30
dtype: int64

In [ ]:
# Identificar outliers
outliers = df[(df['Idade'] < Q1 - 1.5 * IQR) | (df['Idade'] > Q3 + 1.5 * IQR)]

# Imprimir resultados
print("Número de outliers na coluna 'Idade':", len(outliers))

# Salvar os outliers em um arquivo CSV
outliers.to_csv('outliers.csv', index=False)

### **Cross Validation com 10 algoritmos**

### Leitura do arquivo csv

In [9]:
dados = pd.read_csv('Casos_e_obitos_ESP.csv', sep=';')

,AM or PM,Lot
0,AM,4932
1,PM,5068


### Remoção de linhas com valores faltantes

In [ ]:
dados.dropna(inplace=True)

### Seleção de atributos (x) e variável alvo (y)

In [11]:
X = dados[['Asma', 'Cardiopatia', 'Data_Inicio_Sintomas', 'Diabetes', 'Diagnostico_Covid19', 'Doenca_Hematologica',
           'Doenca_Hepatica', 'Doenca_Neurologica', 'Doenca Renal', 'Genero', 'Idade', 'Imunodepressao', 'Municipio',
           'Obesidade', 'Outros_Fatores_De_Risco', 'Pneumopatia', 'Puérpera', 'Síndrome_De_Down']]
y = dados['Obito']

513    75.1
Name: Purchase Price, dtype: float64

### Inicialização de um codificador de rótulos

In [12]:
le = LabelEncoder()

1234    bondellen@williams-garza.com
Name: Email, dtype: object

### Normalização dos dados

In [ ]:
# Codificar variáveis categóricas
for column in X.columns:
    if X[column].dtype == 'object':
        X.loc[:, column] = le.fit_transform(X[column])

In [ ]:
# Inicializar o escalador Min-Max
scaler = MinMaxScaler()

In [ ]:
# Normalizar os dados
X_normalizado = scaler.fit_transform(X)

### Lista de classificadores

In [16]:
classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    KNeighborsClassifier(),
    SVC(),
    LogisticRegression(),
    GradientBoostingClassifier(),
    XGBClassifier(),
    LGBMClassifier(),
    CatBoostClassifier(verbose=False),
    LogisticRegression(multi_class='multinomial', solver='lbfgs')
]

1033

### Dicionário para armazenar as métricas de cada classificador

In [ ]:
metrics = {
    'Acurácia': [],
    'Precisão': [],
    'Sensibilidade/Recall': [],
    'Medida-F1': []
}

### Execução da cross validation para cada classificador

In [ ]:
for classifier in classifiers:
    # Calcula a acurácia usando validação cruzada (cv=10 para 10 folds)
    accuracy = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='accuracy').mean()
    # Calcula a precisão usando validação cruzada
    precision = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='precision').mean()
    # Calcula a sensibilidade/recall usando validação cruzada
    recall = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='recall').mean()
    # Calcula a medida-F1 usando validação cruzada
    f1 = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='f1').mean()

### Armazenamento das métricas calculadas no dicionário de métricas

In [ ]:
metrics['Acurácia'].append(accuracy)
metrics['Precisão'].append(precision)
metrics['Sensibilidade/Recall'].append(recall)
metrics['Medida-F1'].append(f1)

### Exibição dos resultados

In [ ]:
for metric, values in metrics.items():
    print(f'{metric}: {values}')